## Reproducing the correlation anlaysis of the evolutionary expansion map to various brain maps

In [1]:
import neuromaps
print("neuromaps version:", neuromaps.__version__)

neuromaps version: 0.0.5+41.gf0ed67c


#### *Make sure to have Connectome Workbench installed*

In [ ]:
## run if needed
# pip install neuromaps brainspace
# pip install statsmodels

In [2]:
# Add more imports if needed

from neuromaps.datasets import fetch_atlas ## used to access the templates for the coordinate system
import nibabel as nib ## used to load system dictionary per key
from neuromaps.datasets import available_annotations ## repository of brain maps - spatial maps representing some
from neuromaps.datasets import available_tags ## most annotations have “tags” that help to describe the data they represent
from neuromaps.datasets import fetch_annotation
from neuromaps.datasets import fetch_fsaverage

from neuromaps import transforms
import netneurotools
# possibly need
from netneurotools import datasets as nntdata
from neuromaps import parcellate
from neuromaps.parcellate import Parcellater
from neuromaps.images import dlabel_to_gifti
# plotting 
from neuromaps.images import load_data
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from neuromaps import plotting
from nilearn import plotting
import numpy as np
import pandas as pd
# sampling
from neuromaps import datasets, images, nulls, resampling
from neuromaps.resampling import resample_images
from neuromaps.stats import compare_images
from neuromaps import stats
from nilearn.datasets import fetch_atlas_surf_destrieux
from neuromaps.nulls import alexander_bloch
from neuromaps.stats import compare_images
from scipy.stats import pearsonr

from nilearn.surface import load_surf_mesh
from brainspace.null_models import SpinPermutations
from nilearn.surface import InMemoryMesh, PolyMesh
from nilearn.surface import SurfaceImage
from nilearn.plotting import view_surf

import time
# from neuromaps.stats import fdr_correct
# for FDR
from statsmodels.stats.multitest import multipletests

#### Our Source Map

In [ ]:
# Note: source_map will be used later in the full_spin_test function
source_map = {'source':'hill2010', 'desc':'evoexp', 'space':'fsLR', 'den':'164k'}
# Demonstration on fetching the path to the data map file that is used in the full_spin_test function shown later
evolutionary_expansion_map = fetch_annotation(**source_map)
evolutionary_expansion_map

Ellipsis

#### Our Target Maps

In [ ]:
# Note: these will be used later in the full_spin_test function
target_maps = [
    {'source':'abagen', 'desc':'genepc1', 'space':'fsaverage', 'den':'10k'},
    {'source':'hcps1200', 'desc':'myelinmap', 'space':'fsLR', 'den':'32k'},
    {'source':'hcps1200', 'desc':'thickness', 'space':'fsLR', 'den':'32k'},
    {'source':'hill2010', 'desc':'devexp', 'space':'fsLR', 'den':'164k'},
    {'source':'margulies2016', 'desc':'fcgradient01', 'space':'fsLR', 'den':'32k'},
    {'source':'mueller2013', 'desc':'intersubjvar', 'space':'fsLR', 'den':'164k'},

    {'source':'raichle', 'desc':'cbf', 'space':'fsLR', 'den':'164k'},
    {'source':'raichle', 'desc':'cbv', 'space':'fsLR', 'den':'164k'},
    {'source':'raichle', 'desc':'cmr02', 'space':'fsLR', 'den':'164k'},
    {'source':'raichle', 'desc':'cmrglc', 'space':'fsLR', 'den':'164k'},
    {'source':'reardon2018', 'desc':'scalingnih', 'space':'civet', 'den':'41k'},
    {'source':'reardon2018', 'desc':'scalingpnc', 'space':'civet', 'den':'41k'}
]

# Official names for plotting later
map_names = {
    'genepc1': 'PC1 Gene Expression',
    'myelinmap': 'T1w/T2w Ratio',
    'thickness': 'Cortical Thickness',
    'devexp': 'Developmental Expansion',
    'fcgradient01': 'Functional Gradient',
    'intersubjvar': 'Intersubject Variability',
    'cbf': 'Cerebral Blood Flow',
    'cbv': 'Cerebral Blood Volume',
    'cmr02': 'Oxygen Metabolism',
    'cmrglc': 'Glucose Metabolism',
    'scalingnih': 'Allometric Scaling (NIH)',
    'scalingpnc': 'Allometric Scaling (PNC)',
    'evoexp': 'Evolutionary Expansion'
}

#### Implementing the Spin Test for our Target Maps

Also demonstrates transforming brain maps to different coordinate systems and densities

In [ ]:
def full_spin_test(src: dict, trg: dict):
    """
    Performs the Alexander Bloch spin test between the source map and target map
    Parameters:
    src: the source map as a dictionary with the needed parameters for feth_annotation
    trg: the target maps as a dictionary with the needed parameters for feth_annotation
    Outputs: A list of dictionaries 
    """
    src_paper, src_desc, src_space, src_den = src.values()
    trg_paper, trg_desc, trg_space, trg_den = trg.values()

    #fetch source map and target map files
    start = time.perf_counter() #timing just for user info
    src_map = datasets.fetch_annotation(**src)
    trg_map = datasets.fetch_annotation(**trg)

    #if target map has both hemispheres, use only the right one
    #our source map only contains data for the right hemisphere
    if(len(trg_map)==2):
        trg_map = trg_map[1]
    if src_den != trg_den:
        src_res, trg_res = resampling.resample_images(
            src_map,
            trg_map,
            src_space=src_space,
            trg_space=trg_space,
            hemi='R',
            resampling='downsample_only' #sample to size of target map density
        )
        #load the actual data from the files
        src_data = images.load_data(src_res)
        trg_data = images.load_data(trg_res)
    else:
        src_data = images.load_data(src_map)
        trg_data = images.load_data(trg_map)
    #create nan values for left brain
    L_nan = np.full_like(src_data, np.nan)
    src_sphere = np.hstack([L_nan, src_data])
    trg_sphere = np.hstack([L_nan, trg_data])
    #spin test
    rotated = nulls.alexander_bloch(
        src_sphere,
        atlas = trg_space,
        density = trg_den,
        n_perm = 1000,
        seed = 1234
    )
    r_corr, p_value, null_dist = compare_images(
        src_sphere,
        trg_sphere,
        metric='pearsonr',
        nulls=rotated,
        return_nulls=True
    )
    end = time.perf_counter()
    time_elapsed = end - start
    print(f"Time elapsed for {trg_desc}: {time_elapsed/60:.2f} minutes")
    
    formal_name = map_names.get(trg_desc)
    results_dict = {'target map': formal_name, 
                'r_emp': r_corr, 
                'p_spin': p_value, 
                'nulls': null_dist
               }
    return results_dict

In [ ]:
results = []
for trg_map in target_maps:
    results.append(full_spin_test(source_map, trg_map))

results_df = pd.DataFrame(results)
display(results_df)

#### False Discovery Rate

In [ ]:
## False Discovery Rate Calculation
## Multiple Comparison Tests

p_vals = np.array(results_df['p_spin'])

# Benjamini–Hochberg FDR
reject_fdr, p_fdr, _, _ = multipletests(p_vals, alpha=0.05, method='fdr_bh')
# Bonferroni correction
reject_bonf, p_bonf, _, _ = multipletests(p_vals, alpha=0.05, method='bonferroni')

results_df['p_FDR'] = p_fdr
results_df['reject_FDR'] = reject_fdr
results_df['p_Bonf'] = p_bonf
results_df['reject_Bonf'] = reject_bonf

summary_df = results_df[['target map', 'p_spin', 'p_FDR', 'reject_FDR', 'p_Bonf', 'reject_Bonf']].copy()
summary_df.columns = [
    'Target map',
    'p_spin (uncorrected)',
    'p_FDR (BH corrected)',
    'Significant (FDR<0.05)',
    'p_Bonferroni (corrected)',
    'Significant (Bonf<0.05)'
]
summary_df = summary_df.round(4)

print("Summary of results:")
display(summary_df)

Ellipsis

### Plotting Box Plot
Our findings

In [ ]:
## Code for plotting box plots for each target map

results_df = pd.DataFrame(results)
box_data = [np.ravel(np.array(n)) for n in results_df['nulls']]
positions = np.arange(1, len(box_data) + 1)

fig, ax = plt.subplots(figsize=(11, 8))
#boxplots for target maps null distributions
bp = ax.boxplot(
    box_data,
    positions=positions,
    widths=0.6,
    patch_artist=True,
    boxprops=dict(facecolor='white', edgecolor='black', linewidth=1),
    medianprops=dict(color='black', linewidth=1),
    whiskerprops=dict(color='black'),
    capprops=dict(color='black'),
    flierprops=dict(marker='o', color='gray', markersize=3, alpha=0.4)
)

#add spin test r correlation values
for i, (r, p) in enumerate(zip(results_df['r_emp'], results_df['p_spin'])):
    color = 'red' if p < 0.05 else '#e6a67a'  # red = significant, orange = non-significant
    ax.scatter(
        positions[i],
        r,
        color=color,
        s=80,
        edgecolor='black',
        linewidth=0.5,
        zorder=3
    )

ax.axhline(0, color='gray', linestyle='--', linewidth=1)
ax.set_xticks(positions)
ax.set_xticklabels(results_df['target map'], rotation=40, ha='right', fontsize=10)
ax.set_ylabel("Pearson's r", fontsize=12)
ax.set_xlabel("Target maps", fontsize=12)
ax.set_ylim(-0.8, 0.8)
ax.set_title("Empirical correlations vs spatial nulls", fontsize=13, pad=15)

#legend
legend_elements = [
    Line2D([0], [0], marker='o', color='w',
           label=r'Empirical ($P_{spin} ≥ 0.05$)',
           markerfacecolor='#e6a67a', markeredgecolor='black', markersize=8),
    Line2D([0], [0], marker='o', color='w',
           label=r'Empirical ($P_{spin} < 0.05$)',
           markerfacecolor='red', markeredgecolor='black', markersize=8),
    Line2D([0], [0], color='black', lw=1, label='Spatial null')
]
ax.legend(handles=legend_elements, loc='upper right', frameon=False)
plt.show()

#### Target Maps

In [ ]:
## dictionary for target maps plotting

brain_map_settings = {
    'evoexp': {'cmap': 'inferno', 'vmin': -2.7, 'vmax': 2.7},

    'genepc1': {'cmap': 'inferno', 'vmin': -2.7, 'vmax': 2.7},
    'myelinmap':{'cmap': 'viridis', 'vmin': None, 'vmax': None},
    'thickness': {'cmap': 'viridis', 'vmin': None, 'vmax': None},
    'devexp': {'cmap': 'inferno', 'vmin': -1, 'vmax': 1},
    'fcgradient01': {'cmap': 'viridis', 'vmin': None, 'vmax': None},
    'intersubjvar': {'cmap': 'inferno', 'vmin': None, 'vmax': None},
    
    'cbf': {'cmap': 'inferno', 'vmin': None, 'vmax': None}, # need to change
    'cbv': {'cmap': 'inferno', 'vmin': None, 'vmax': None},
    'cmr02': {'cmap': 'inferno', 'vmin': None, 'vmax': None}, # need to change
    'cmrglc': {'cmap': 'inferno', 'vmin': None, 'vmax': None}, # need to change
    'scalingnih': {'cmap': 'inferno', 'vmin': None, 'vmax': None},
    'scalingpnc': {'cmap': 'inferno', 'vmin': None, 'vmax': None},
}

In [ ]:
def plot_brain_map(map: dict, map_names: dict, brain_map_settings: dict):
    """
    Plots the brain map given in the map dictionary
    Parameters:
    map: the map as a dictionary with the needed parameters for feth_annotation
    map_names: dictionary of formal names for readability and plot titles
    Outputs: 
        ...
    """
    start_time  = time.perf_counter() #timing just for user info

    map_paper, map_desc, map_space, map_den = map.values()
    #fetch source map and target map files
    src_map = datasets.fetch_annotation(**map)

    settings = brain_map_settings.get(map_desc, {})
    cmap = settings.get('cmap', 'inferno')

    fig = plt.figure(figsize=(10, 4))
    fslr = fetch_atlas(map_space, map_den)
    surf_mesh_left = fslr['inflated'].L
    surf_mesh_right = fslr['inflated'].R
    data_full = load_data(src_map)

    if settings.get('vmin') is not None and settings.get('vmax') is not None:
        vmin, vmax = settings['vmin'], settings['vmax']
    else:
        vmin, vmax = np.percentile(data_full[~np.isnan(data_full)], [2, 98])

    #if target map has both hemispheres, plot both hemispheres lateral
    if(len(src_map)==2):
        data_l = load_data(src_map[0])
        data_r = load_data(src_map[1])
        ax1 = fig.add_subplot(1, 2, 1, projection='3d')
        plotting.plot_surf(
            surf_mesh=surf_mesh_left,
            surf_map=data_l,
            hemi='left',
            view='lateral',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax,
            colorbar=False,
            axes=ax1,
            title='Left hemisphere'
        )
        # right hemi
        ax2 = fig.add_subplot(1, 2, 2, projection='3d')
        plotting.plot_surf(
            surf_mesh=surf_mesh_right,
            surf_map=data_r,
            hemi='right',
            view='lateral',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax,
            colorbar=False,
            axes=ax2,
            title='Right hemisphere'
        )
        # color bar
        sm = plt.cm.ScalarMappable(cmap=cmap)
        sm.set_clim(vmin, vmax)
        cbar = fig.colorbar(sm, ax=[ax1, ax2], shrink=0.6, location='right')
        cbar.set_label(f"{map_names.get(map_desc)}({map_space} {map_den})", fontsize=11)
        plt.suptitle(f"{map_names.get(map_desc)}", fontsize=14)
        plt.show()


    else: #if only one hemisphere, only plot that hemisphere lateral and medial
        plot_kwargs = dict(
            surf_mesh=surf_mesh_right,
            surf_map=data_full,
            hemi='right',
            bg_on_data=True,
            cmap=cmap,
            vmin=vmin,
            vmax=vmax,
            colorbar=False
        )
        #plot right veiw
        ax1 = fig.add_subplot(1, 2, 1, projection='3d')
        plotting.plot_surf(
            view='lateral',
            axes=ax1,
            **plot_kwargs
        )
        #plot from left view
        ax2 = fig.add_subplot(1, 2, 2, projection='3d')
        plotting.plot_surf(
            view='medial',
            axes=ax2,
            **plot_kwargs
        )
        #color bar
        sm = plt.cm.ScalarMappable(cmap=cmap)
        sm.set_clim(vmin, vmax)
        cbar = fig.colorbar(sm, ax=[ax1, ax2], shrink=0.6, location='right')
        cbar.set_label(f"{map_names.get(map_desc)}({map_space} {map_den})", fontsize=11)
        plt.suptitle(f"{map_names.get(map_desc)}", fontsize=14)
        plt.show()


    end_time = time.perf_counter()
    print(f"Total time for plotting: {(end_time - start_time):.2f} seconds")

    return None

In [ ]:
plot_brain_map(source_map, map_names, brain_map_settings)

In [ ]:
for trg_map in target_maps:
    plot_brain_map(trg_map, map_names, brain_map_settings)